In [1]:
import pandas as pd
import numpy as np

In [2]:
institutions = pd.read_csv("D:/Institutions.csv", encoding='latin-1')

In [19]:
institutions.shape

(1128543, 28)

In [16]:
institutions.columns

Index(['factset_entity_id', 'quarter', 'rquarter', 'ENTITY_PROPER_NAME',
       'CUSIP', 'TIC', 'mktcap', 'io_usd', 'nbr_firms', 'io', 'io_dom',
       'io_for', 'io_cat1', 'io_cat2', 'io_cat3', 'io_cat4', 'io_cat5',
       'io_cat6', 'io_cat7', 'io_indep', 'ibh_5pct', 'ibh_1pct', 'top5',
       'herf'],
      dtype='object')

In [135]:
# Let's see how many US stocks are in the data set

# CUSIP  code for us stocks contains only numbers

#num_cusip = []

#for i in range(0,len(institutions)):
#        num_cusip.append(str.isdigit(str(institutions['CUSIP'][i]))) #I fianlly get it. Append makes a big diff sometimes

In [136]:
#only_US = institutions[num_cusip]

#len(only_US['CUSIP'].unique())

# 8899 stocks is a lot. Its safe to assume we have institutional data for all US stocks. we are going to use this data set then

8899

In [137]:
#only_US.shape

(232844, 27)

In [3]:
financials = pd.read_csv("D:/Financials.csv", encoding='latin-1')

In [20]:
financials.shape

# So the financials data is nearly 3.5 times bigger than institutional data

(826815, 33)

In [52]:
financials.columns

Index(['FS_PERM_SEC_ID', 'CUSIP', 'TIC', 'EXCOUNTRY', 'PRIMARY_SIC_CODE',
       'ZIP_POSTAL_CODE', 'FF_COMPACT', 'FF_MKT_VAL_CURR', 'FF_IS_ADR',
       'FF_SECACT', 'DATE', 'FF_COM_SHS_OUT_EPS_DIL', 'FF_RECEIV_ST',
       'FF_CURR_RATIO', 'FF_GROSS_MGN', 'FF_NET_MGN', 'FF_ROA', 'FF_ROE',
       'FF_ROTC', 'FF_PBK', 'FF_LTD_COM_EQ', 'FF_SALES_PS_GR', 'FF_TAX_RATE',
       'FF_ZSCORE', 'FF_PE_DIL', 'FF_PFCF_DIL', 'FF_PSALES_DIL'],
      dtype='object')

In [67]:
# i wonder if it is possible to join financials and institutions data sets on CUSIP and date?

financials['DATE'][1:5]

1    19980630
2    19980930
3    19990331
4    19990630
Name: DATE, dtype: int64

In [55]:
institutions['quarter'][1:5]

1    200302
2    200303
3    200304
4    200401
Name: quarter, dtype: int64

In [56]:
institutions['rquarter'][1:5]

1    06/30/2003
2    09/30/2003
3    12/31/2003
4    03/31/2004
Name: rquarter, dtype: object

In [5]:
financials['month1'] = financials['DATE'].astype(str).str[4]
financials['month2'] = financials['DATE'].astype(str).str[5]
financials['month'] = financials['month1'] + financials['month2']

In [21]:
financials2 = financials
financials2['month'] = financials2['month'].astype(str).astype(int)
typical_quarter_end = [3,6,9,12]
financials2 = financials2.loc[financials2['month'].isin(typical_quarter_end)]
financials2.shape

# willing to ignore that 16% of data. Will decide later. With the systems I am trying to build and currently use, Portfolio's can be shuffled only 4 times. Can't make exceptions for some firms. moreover, most big firms follow the typical quarter ending schedule

(707613, 33)

In [7]:
financials['year'] = financials['DATE'].astype(str)
financials['year'] = financials.year.str[0:4]
financials['month'] = financials['month'].astype(str)
financials['date_modified'] = financials['month'] + financials['year']
financials['date_modified'][1:5]

1    61998
2    91998
3    31999
4    61999
Name: date_modified, dtype: object

In [8]:
institutions['month'] = institutions['rquarter'].astype(str)
institutions['month'] = institutions.rquarter.str[0:2]

In [9]:
institutions['year'] = institutions['rquarter'].astype(str)
institutions['year'] = institutions.rquarter.str[6:11]

In [10]:
institutions['date_modified'] = institutions['month'] + institutions['year']

In [127]:
institutions['date_modified'][1:5]

1    062003
2    092003
3    122003
4    032004
Name: date_modified, dtype: object

In [11]:
# forming the real keys to be joined on

financials['key1'] = financials['date_modified'] + financials['CUSIP']

In [129]:
financials['key1'][1:5]

# If i remember right, there are some canadian stocks also in the financials set

1    061998FDSA783E7
2    091998FDSA783E7
3    031999FDSA783E7
4    061999FDSA783E7
Name: key1, dtype: object

In [12]:
financials['EXCOUNTRY'].unique()

# I just relaized that I have data for 53 countries. What a shame we do not have price data. I can scrape but let me just focus on american stocks for now

array([nan, 'OTC', 'NAS', 'NYS', 'TSE', 'ASE', 'TSX', 'LON', 'ASX', 'PHS',
       'SWX', 'PSE', 'MEX', 'FRA', 'BUE', 'ETR', 'CAR', 'OSL', 'AMS',
       'BSP', 'PAR', 'TAI', 'HKG', 'MIC', 'SES', 'CNQ', 'TKS', 'BOM',
       'OME', 'JSE', 'STU', 'GUA', 'MCE', 'WAR', 'MUN', 'MIL', 'TAE',
       'JKT', 'HEL', 'SHE', 'CSE', 'SHG', 'BOG', 'BKK', 'COL', 'ATH',
       'RUS', 'NGM', 'KRX', 'IST', 'NSA', 'BATS', 'MAU'], dtype=object)

In [13]:
usa_markets = ['NAS', 'NYS', 'BATS']

financials_us = financials.loc[financials['EXCOUNTRY'].isin(usa_markets)]
financials_us.shape
# 342757 rows. Sounds right to me. But did I aslo include U.S. bonds? We will know soon

(342757, 33)

In [150]:
len(financials_us['CUSIP'].unique())

# 8492. Seems very reasonable and also in line with what we saw in earlier data set

8492

In [14]:
institutions['key2'] = institutions['date_modified'] + institutions['CUSIP']

institutions['key2'][1:5]

1    062003170386106
2    092003170386106
3    122003170386106
4    032004170386106
Name: key2, dtype: object

In [17]:
financials_institutions_merged = pd.merge(financials_us,
                                          institutions,
                                          left_on='key1',
                                          right_on='key2',
                                          how='left')

In [18]:
financials_institutions_merged.shape

(342757, 61)

In [153]:
financials_institutions_merged.columns

# I just realized that common name columns are renamed as colname_x and colname_y. Thanks Wes Mckinney

Index(['FS_PERM_SEC_ID', 'CUSIP_x', 'TIC_x', 'EXCOUNTRY', 'PRIMARY_SIC_CODE',
       'ZIP_POSTAL_CODE', 'FF_COMPACT', 'FF_MKT_VAL_CURR', 'FF_IS_ADR',
       'FF_SECACT', 'DATE', 'FF_COM_SHS_OUT_EPS_DIL', 'FF_RECEIV_ST',
       'FF_CURR_RATIO', 'FF_GROSS_MGN', 'FF_NET_MGN', 'FF_ROA', 'FF_ROE',
       'FF_ROTC', 'FF_PBK', 'FF_LTD_COM_EQ', 'FF_SALES_PS_GR', 'FF_TAX_RATE',
       'FF_ZSCORE', 'FF_PE_DIL', 'FF_PFCF_DIL', 'FF_PSALES_DIL', 'month1',
       'month2', 'month_x', 'year_x', 'date_modified_x', 'key1', 'final_key1',
       'final_key2', 'factset_entity_id', 'quarter', 'rquarter',
       'ENTITY_PROPER_NAME', 'CUSIP_y', 'TIC_y', 'mktcap', 'io_usd',
       'nbr_firms', 'io', 'io_dom', 'io_for', 'io_cat1', 'io_cat2', 'io_cat3',
       'io_cat4', 'io_cat5', 'io_cat6', 'io_cat7', 'io_indep', 'ibh_5pct',
       'ibh_1pct', 'top5', 'herf', 'month_y', 'year_y', 'date_modified_y',
       'key2'],
      dtype='object')

In [22]:
len(financials_institutions_merged['CUSIP_x'].unique())

8492

In [23]:
len(financials_institutions_merged['CUSIP_y'].unique())

# I remember that the institution data set starts from 2000. However, the financial data set starts from 1990

5190

In [ ]:
# So there are 8492 CUSIP's. That's great. But, for how many stocks do we have atleast 4 values (4 quarters is 1 year)
# If a stock hasn't traded for atleast a year, i think its best if I just drop that CUSIP right now

In [178]:
CUSIP_count_df = financials_institutions_merged[['CUSIP_x','FS_PERM_SEC_ID']].groupby(['CUSIP_x']).agg(['count'])
CUSIP_count_df.columns = ['count']
CUSIP_count_df['CUSIP_stock'] = CUSIP_count_df.index

In [186]:
len(CUSIP_count_df[CUSIP_count_df['count'] < 5])/len(CUSIP_count_df)

# I am so relieved to know that barely 5% of stocks in the data traded for less than a year. let me just get a frequency plot

0.04463024022609515

In [33]:
# loading in the monthly price and volume data set

price_volume = pd.read_csv("D:/Monthly price and volume.csv", encoding='latin-1', low_memory = False)

In [34]:
price_volume.shape

(2527070, 23)

In [35]:
price_volume[1:5]

,PERMNO,date,SHRCD,SICCD,COMNAM,SHRCLS,TSYMBOL,TRDSTAT,SECSTAT,PERMCO,...,ASKHI,PRC,VOL,RET,SHROUT,CFACPR,CFACSHR,vwretd,ewretd,sprtrn
1,10001,02/28/1990,11.0,4920,GREAT FALLS GAS CO,NaN,GFGC,A,R,7953,...,10.000,-9.875,149.0,-0.006289,1022.0,3.0,3.0,0.014901,0.015434,0.008539
2,10001,03/30/1990,11.0,4920,GREAT FALLS GAS CO,NaN,GFGC,A,R,7953,...,10.125,-9.875,127.0,0.012658,1027.0,3.0,3.0,0.024140,0.021315,0.024255
3,10001,04/30/1990,11.0,4920,GREAT FALLS GAS CO,NaN,GFGC,A,R,7953,...,10.000,-9.875,166.0,0.000000,1027.0,3.0,3.0,-0.028286,-0.028116,-0.026887
4,10001,05/31/1990,11.0,4920,GREAT FALLS GAS CO,NaN,GFGC,A,R,7953,...,10.000,9.750,279.0,-0.012658,1027.0,3.0,3.0,0.088934,0.045673,0.091989


In [41]:
CUSIP_list = price_volume['CUSIP']
CUSIP_list = list(set(CUSIP_list))
len(CUSIP_list)

23694

In [46]:
# just of curiousity, how many unique PERM_NO are there?

PERMNO_list = price_volume['PERMNO']
PERMNO_list = list(set(PERMNO_list))
len(PERMNO_list)

# I know that 1 PERMNO can have multiple CUSIP. yet interestingly, both have same number of uniques

23694

In [ ]:
# its interesting that there are nearly 23694 CUSIP's though. 

# Were there 23,000 stocks since 1990. I don't think so. If there were that many, I have the financials data only for roughly 8000
# My guess is that those extra 15000 stocks are OTC, ADR, candian or penny stocks etc. Let me confirm that

In [107]:
CUSIP_list2 = CUSIP_list[2]

In [135]:
CUSIP_list2

'26927920'

In [188]:
type(CUSIP_list)

str

In [189]:
for i in CUSIP_list2:
    print(i)
    
# What's going on here? Didn't expect this. Really sneaky error which I found out further down wrangling

2
6
9
2
7
9
2
0


In [149]:
PERMNO_list2 = PERMNO_list[2:4]

In [150]:
PERMNO_list2

[10003, 10005]

In [151]:
for i in PERMNO_list2:
    print(i)

# Amazing. It works fine with PERMNO though. 

10003
10005


In [153]:
price_volume.dtypes

# And that's why they say to set the type if column is too important!

PERMNO       int64
date        object
SHRCD      float64
SICCD       object
COMNAM      object
SHRCLS      object
TSYMBOL     object
TRDSTAT     object
SECSTAT     object
PERMCO       int64
CUSIP       object
NWPERM     float64
BIDLO      float64
ASKHI      float64
PRC        float64
VOL        float64
RET         object
SHROUT     float64
CFACPR     float64
CFACSHR    float64
vwretd     float64
ewretd     float64
sprtrn     float64
dtype: object

In [ ]:
# The most important thing is to convert the entire monthly price volume data set into quarterly paradigm

# Before we do that lets play with some columns to ensure I am manipualting them correctly when I do the conversion process

In [163]:
# For most of the columns we only expect positive values.

# Broadly, for some of the columns a negative value or zero indicates something. So, how many special situations are there across columns
# For more specific details, refer to data dictionary

def bad_data(column_name):
    nulls = len(price_volume[price_volume[column_name] == None])/len(price_volume)
    zeros = len(price_volume[price_volume[column_name] == 0])/len(price_volume)
    negatives = len(price_volume[price_volume[column_name] < 0])/len(price_volume)
    special_situations = pd.Series([nulls, zeros, negatives])
    return special_situations

In [164]:
print(bad_data('VOL'))
print(bad_data('ASKHI'))
print(bad_data('PRC'))
print(bad_data('BIDLO'))
print(bad_data('RET'))
print(bad_data('SHROUT'))

0    0.000000
1    0.001427
2    0.000000
dtype: float64
0    0.000000
1    0.000000
2    0.050711
dtype: float64
0    0.000000
1    0.000000
2    0.069031
dtype: float64
0    0.000000
1    0.000000
2    0.039084
dtype: float64


TypeError: '<' not supported between instances of 'str' and 'int'

In [ ]:
print(bad_data('SHROUT'))

In [162]:
# The most important thing is to convert the entire monthly price volume data set into quarterly paradigm

all_stocks = price_volume[1:1] # nice way to set all stocks df the columns of price_volume

for i in PERMNO_list:
    individual_stock = price_volume[price_volume['PERMNO'] == i]
    vol_num = individual_stock.columns.get_loc("VOL")
    individual_stock['three_month_vol'] = 1
    individual_stock['High'] = individual_stock[]
    three_month_num = individual_stock.columns.get_loc("three_month_vol")
    for j in range(2,len(individual_stock)):
        individual_stock.iloc[j,three_month_num] = individual_stock.iloc[j, vol_num] + individual_stock.iloc[j-1, vol_num] + individual_stock.iloc[j-2, vol_num]
        individual_stock.iloc[0, three_month_num] = individual_stock.iloc[0, vol_num]*3
        individual_stock.iloc[1, three_month_num] = individual_stock.iloc[1, vol_num]*3
    all_stocks = all_stocks.append(individual_stock, ignore_index=True)    
    
# ignore the warnings below. Its the typical 'a value being set on slice etc.' 

C:\Users\Prasanth\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Prasanth\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


KeyboardInterrupt: 

In [24]:
fund13f = pd.read_csv("D:/13F.csv", encoding='latin-1')

In [ ]:
fund13f.shape

# 6.7 million rows. This data set is much bigger than the previous 2 data sets. But most importantly, it is possible to use columns to join this data set

In [164]:
fund13f.columns

Index(['FACTSET_ENTITY_ID', 'FSYM_ID', 'REPORT_DATE', 'ADJ_MV',
       'REPORTED_HOLDING'],
      dtype='object')

In [165]:
len(fund13f['FACTSET_ENTITY_ID'].unique())

8135

In [166]:
len(fund13f['FSYM_ID'].unique())

# I thought FSYM_ID is the unique key for stocks. But there aren't 59,980 stocks. May be FACTSET_ENTITY_ID is the unique key for stocks

59980

In [167]:
fund13f['FACTSET_ENTITY_ID'][1:5]

1    000BJX-E
2    000BJX-E
3    000BJX-E
4    000BJX-E
Name: FACTSET_ENTITY_ID, dtype: object

In [168]:
fund13f['FSYM_ID'][1:5]

1    B00CQC-S
2    B00CQC-S
3    B00CQC-S
4    B00CQC-S
Name: FSYM_ID, dtype: object

In [171]:
financials_institutions_merged['FS_PERM_SEC_ID'][1:5]

1    T1N9J9-S-US
2    T1N9J9-S-US
3    T1N9J9-S-US
4    T1N9J9-S-US
Name: FS_PERM_SEC_ID, dtype: object

In [172]:
# verified on WRDS forums. FACTSET_ENTITY_ID is the actual unique key for stocks. Is it referred by a different name in the earlier merged data set

stocks_code1 = fund13f['FACTSET_ENTITY_ID']
stocks_code2 = financials_institutions_merged['FS_PERM_SEC_ID']

In [174]:
common_codes1 = pd.Series(list(set(stocks_code1).intersection(set(stocks_code2))))

len(common_codes1)

# But there are no common codes. I think the financials_institutions_merged codes have '-US' at the end. 

0

In [27]:
fund13f['New_FSYM_ID'] = fund13f['FSYM_ID'] + '-US'

In [28]:
fund13f[1:5]

,FACTSET_ENTITY_ID,FSYM_ID,REPORT_DATE,ADJ_MV,REPORTED_HOLDING,New_FSYM_ID
1,000BJX-E,B00CQC-S,03/31/2005,390040.0,9800,B00CQC-S-US
2,000BJX-E,B00CQC-S,06/30/2005,390299.0,9100,B00CQC-S-US
3,000BJX-E,B00CQC-S,09/30/2005,417417.0,9100,B00CQC-S-US
4,000BJX-E,B00CQC-S,12/31/2005,188225.0,4350,B00CQC-S-US


In [30]:
fund13f_stock_codes = fund13f2['New_FSYM_ID']
earlier_merged_stock_codes = financials_institutions_merged['FS_PERM_SEC_ID']

common_codes = pd.Series(list(set(fund13f_stock_codes).intersection(set(earlier_merged_stock_codes))))
len(common_codes)

# yup. Adding 'US' did the trick

# Read the research papers written by Professor Mateos who developed the institutional data set. It appears he is focused on corporate finance related metrics rather qunatitative asset management (or predicting stock market)

7539

In [ ]:
# Now that we can use the 13F data set, the question is how do I calculate 'how good a fund is?'